##### Unit 12 - Tales from the Crypto

---


###### 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [10]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Read your api key environment variable
api_key = os.getenv("news_api")

In [12]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

NameError: name 'NewsApiClient' is not defined

In [4]:
# Fetch the Bitcoin news articles
bitcoin_news = newsapi.get_everything(q='Bitcoin', language='en')

In [5]:
# Fetch the Ethereum news articles
ethereum_news = newsapi.get_everything(q='Ethereum', language='en')

In [ ]:
## Create the Bitcoin sentiment scores DataFrame

def create_sentiment_scores_dataframe(news_headlines):
    '''Capture sentiment scores from news headlines.
    Arguments: 
        news_headlines (dict of dict): Dictionary of dictionaries from NewsApiClient
    Returns:
        A dataframe of sentiment scores with matching article text
    '''

    # Instantiate sentimate list
    news_sentiments = []

    # Loop through news headlines for each article
    for article in news_headlines['articles']:
        try:
            # Assign specific features of each article
            text = article['content']
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment['compound']
            negative = sentiment['neg']
            neutral = sentiment['neu']
            positive = sentiment['neg']

            # Append each articles sentiment scores to a list
            news_sentiments.append({
                'Compound': compound,
                'Negative': negative,
                'Neutral': neutral,
                'Positive': positive,
                'Text': text
            })

        except AttributeError:
            pass

    # Create dataframe
    news_sentiment_df = pd.DataFrame(news_sentiments)

    # Return dataframe
    return news_sentiment_df

bitcoin_df = create_sentiment_scores_dataframe(bitcoin_news)
bitcoin_df.head()6

In [7]:
# Create the Ethereum sentiment scores DataFrame
ethereum_df = create_sentiment_scores_dataframe(ethereum_news)
ethereum_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

In [9]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin has the highest mean positive score: 0.029000.

Q: Which coin had the highest compound score?

A: Ethereum has the highest compound score, max: 0.877900 

Q. Which coin had the highest positive score?

A: Bitcoin has the highest positive score, max: 0.215000.



---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
sw_add_ons = {'', 'chars', 'char', 'volod', 'ruvic', 'reutersdado', 'dado', 'ruvic', 'ruvicillustration', 'volod', 'la', 'im', 'drin', 'wa', 'ha', 'r

# Expand the default stopwords list if necessary
# YOUR CODE HERE!-

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Create a list of the words
    words_tokened = word_tokenize(text)

    # Convert the words to lowercase
    words_lowered = [word.lower() for word in words_tokened]

    # Remove the punctuation
    regex = re.compile('[^a-zA-Z]')
    words_regexed = [regex.sub('', word) for word in words_lowered]
    
    # Remove the stop words
    sw = set(stopwords.words('english'))
    stopwords_removed = [word.lower() for word in words_regexed if word not in sw.union(sw_add_ons)]

    # Lemmatize Words into root words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in stopwords_removed]

    return tokens

In [13]:
# Create a new tokens column for Bitcoin
bitcoin_tokens = [tokenizer(headline) for headline in bitcoin_df['Text']]
bitcoin_df['Tokens'] = bitcoin_tokens
bitcoin_df.head(

In [14]:
# Create a new tokens column for Ethereum
ethereum_tokens = [tokenizer(headline) for headline in ethereum_df['Text']]
ethereum_df['Tokens'] = ethereum_tokens
ethereum_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
def join_df_column_to_string(dataframe_column):
    '''Joins dataframe column of strings and returns one big string.'''
    new_big_string = [' '.join(item) for item in dataframe_column]
    return new_big_string

In [17]:
# Generate the Ethereum N-grams where N=2
ethereum_big_string = join_df_column_to_string(ethereum_df['Tokens'])

ethereum_bigrams = get_bigrams(ethereum_big_string)

ethereum_frequency = Counter(ethereum_bigrams).most_common(20)
ethereum_frequency

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
bitcoin_big_string = ' '.join(bitcoin_df['Text'])
bitcoin_words_tokened = tokenizer(bitcoin_big_string)
bitcoin_word_count = token_count(bitcoin_words_tokened)
bitcoin_word_count

In [20]:
# Use token_count to get the top 10 words for Ethereum
ethereum_big_string = ' '.join(ethereum_df['Text'])
ethereum_words_tokened = tokenizer(ethereum_big_string)
ethereum_word_count = token_count(ethereum_words_tokened)
ethereum_word_count

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
bitcoin_words_joined = ' '.join(bitcoin_words_tokened)
wc = WordCloud().generate(bitcoin_words_joined)
plt.imshow(wc)
plt.title('Bitcoin Word Cloud',
    fontdict={'size': 35},
    loc='Center'
)
plt.axis('off')

###### Generate the Ethereum word cloud
ethereum_words_joined = ' '.join(ethereum_words_tokened)
wc = WordCloud().generate(ethereum_words_joined)
plt.imshow(wc)
plt.title('Ethereum Word Cloud',
    fontdict={'size': 35},
    loc='Center'
)
plt.axis('off')6

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
bitcoin_big_string

In [28]:
# Run the NER processor on all of the text
bitcoin_ner_doc = nlp(bitcoin_big_string)

# Add a title to the document
bitcoin_ner_doc.user_data['title'] = 'Bitcoin NER'

In [29]:
# Render the visualization
displacy.render(bitcoin_ner_doc, style='ent')

In [30]:
# List all Entities
for ent in bitcoin_ner_doc.ents:
    print(ent.text, ent.label_)

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
ethereum_big_string

In [32]:
# Run the NER processor on all of the text
ethereum_ner_doc = nlp(ethereum_big_string)

# Add a title to the document
ethereum_ner_doc.user_data['title'] = 'Ethereum NER'

In [33]:
# Render the visualization
displacy.render(ethereum_ner_doc, style='ent')

In [34]:
# List all Entities
for ent in ethereum_ner_doc.ents:
    print(ent.text, ent.label_)

---